In [1]:

import assemblyai as aai
from dotenv import load_dotenv
import os
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import ast
import torch

load_dotenv()

aai.settings.api_key = os.getenv("AS_API_KEY")

In [2]:
from model_def import *
from train_script import *
from save_load import *
from test_inference import *

In [3]:
creds_json = '../g_sheet_auth.json'
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_json, scope)
client = gspread.authorize(creds)

sheet_url = 'Grind Sheet v2'
sheet = client.open(sheet_url).sheet1  
df = pd.DataFrame(sheet.get_all_records())

In [4]:
extract_index = 0
save_path = "../../saved_models/V2/20-ep/20-ep.pt"

hyperparameters = df.iloc[0]
print(hyperparameters)
encoder_dict = ast.literal_eval(hyperparameters['Encoder Dict'])
end_pred_sequential = eval(hyperparameters['FFNN'])
same_encoder = True if hyperparameters['Same Eno'].lower() == "true" else False
encoder_grad = True if hyperparameters['Encoder Req Grad'].lower() == "true" else False
concat_method = eval(hyperparameters["Concat Method"])
tokenizer = AutoTokenizer.from_pretrained(encoder_dict['path'])
encoder = AutoModel.from_pretrained(encoder_dict['path'])

Test Group                                                    Control
Independent Var                                                    NA
Complete                                                         TRUE
Data Points                                                     50000
Batch Size                                                         64
Encoder Dict        {'path':'sentence-transformers/all-MiniLM-L6-v...
Same Eno                                                         TRUE
Encoder Req Grad                                                FALSE
Concat Method                                       ("u","v","|u-v|")
FFNN                nn.Sequential(    \n    nn.Linear(in_features=...
Epochs                                                             40
Freeze Epochs                                                    None
lr                                                              0.001
Train Loss                                                      0.062
Test Loss           

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'


# model = TSN_model(encoder_model=encoder_dict['path'], 
#                   context_vector_len=encoder_dict['size'], 
#                   device=device, 
#                   same_encoder=same_encoder,
#                   encoder_tune=encoder_grad,
#                   end_pred_sequential=end_pred_sequential,
#                   concat_method=concat_method).to(device)

In [6]:
final_unit = nn.Sequential(    
    nn.Linear(in_features=3*encoder_dict['size'], out_features=3*encoder_dict['size']),
    nn.BatchNorm1d(3*encoder_dict['size']),
    nn.ReLU(),
    nn.Linear(in_features=3*encoder_dict['size'], out_features=512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Linear(256, 1)
)
post_path = "../../saved_models/Control/FFNN.pt"
post_e_model = load_post_encoder(model=final_unit, path=post_path)
print(post_e_model)

Sequential(
  (0): Linear(in_features=1152, out_features=1152, bias=True)
  (1): BatchNorm1d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=1152, out_features=512, bias=True)
  (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=512, out_features=256, bias=True)
  (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Linear(in_features=256, out_features=1, bias=True)
)


In [28]:
def mean_pooling(model_output, attention_mask):
        """
        Applies mean pooling to the model output.

        Args:
            model_output: The output from the model.
            attention_mask: The attention mask.

        Returns:
            torch.Tensor: The pooled output.
        """
        #token_embeddings = model_output[0] First element of model_output contained all token embeddings grabed in embed_reshape
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(model_output.size()).float()
        return torch.sum(model_output * input_mask_expanded, -2) / torch.clamp(input_mask_expanded.sum(-2), min=1e-9)  
      
def embed_str(string:str):
    tok = tokenizer(string, padding=True, truncation=True, return_tensors='pt')
    encoding = encoder(**tok)
    return mean_pooling(encoding[0], tok['attention_mask'])


In [29]:
encode_thres = embed_str(threshold)
encode_test = embed_str(test_caption)

def compute_bool(model, encode_cap, encode_thres, threshold:int=.5):
    with torch.inference_mode():
        model.eval()
        input = torch.cat((encode_cap, encode_thres, torch.abs(encode_cap-encode_thres)), dim=-1)
        logit = model(input)
        prob = torch.sigmoid(logit).item()
        return True if prob > threshold else False

compute_bool(post_e_model, encode_test, encode_thres)

False

In [30]:
import subprocess
import yagmail

def send_message():
    
    sender_email = "stanloosmore@gmail.com"
    app_password = "ebfegwqbvuyosilm"

    try:
        # initializing the server connection
        yag = yagmail.SMTP(user=sender_email, password=app_password)

        # sending the email
        yag.send(to="loosmore@usc.edu", 
                subject="Demo fired1", 
                contents="This is my website: withode.com")

        print("Email sent successfully")
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
    # applescript = f'''
    # tell application "Messages"
    #     set targetService to 1st service whose service type = iMessage
    #     set targetBuddy to buddy "{phone_number}" of targetService
    #     send "{message}" to targetBuddy
    # end tell
    # '''
    
    # subprocess.run(["osascript", "-e", applescript])

In [25]:
threshold = "agree to email a website"
test_caption = "let me do the demo"

In [44]:
# phone_number = "2069799210"
# baylor_num = "4159994536"
# message='withode.com'
triggered = False

In [32]:
def on_data(transcript: aai.RealtimeTranscript):
  "This function is called when a new transcript has been received."
  global triggered

  if not transcript.text:
    return

  if isinstance(transcript, aai.RealtimeFinalTranscript):
    caption = transcript.text
    print(caption, end="\r\n")
    encode_cap = embed_str(caption)
    is_satisfied = compute_bool(post_e_model, encode_cap, encode_thres)
    if is_satisfied and not triggered:
      print("SENDING MESSAGE", end="\r\n")
      send_message()    # run it throught the pipeline
      triggered = True
    else:
      print('not met')
  else:
    print(transcript.text, end="\r")


In [45]:
def on_error(error: aai.RealtimeError):
  print("An error occured:", error)

def on_open(session_opened: aai.RealtimeSessionOpened):
  "This function is called when the connection has been established."

  print("Session ID:", session_opened.session_id)

transcriber = aai.RealtimeTranscriber(
  on_data=on_data,
  on_error=on_error,
  sample_rate=44_100,
  on_open=on_open, # optional
)

In [46]:
transcriber.connect()
# Open a microphone stream
microphone_stream = aai.extras.MicrophoneStream()

# Press CTRL+C to abort
transcriber.stream(microphone_stream)

transcriber.close()

Session ID: dfbebe1f-0055-4064-bcf8-0724b4121b75
Hey, I'm Stan, and I'll be giving a brief demo.
not met
So if you remember, my whole thing was around.
not met
Threshold detection and using natural language.
not met
To figure out.
not met
If something was met or something was inside of another thing.
not met
So now I'll actually give you an example of that.
not met
And so let's say we're in a conversation.
not met
And you were like, hey, Stan, I'm really curious about what you've been working on.
not met
Can you give me some more information?
not met
And I would say.
not met
Great. Let me send you my website.
SENDING MESSAGE
Email sent successfully
And then it would be sent to you.
not met
Like, how cool would that be? Well.
not met
I just haven't. So let me show you what this means.
not met
If we go into here. 657. 657.seven six fifty seven
not met
So I'll tell you. I hard coded it to send you a website.
not met
Because that's not what I'm working on. What I'm working on specifically.